In [ ]:
data = {
    '/home/gisuser/autocls2/fmask2018/fmask_outono_source.tif' : [
        '/home/gisuser/autocls2/i_20170921',
        '/home/gisuser/autocls2/i_20170926',
        '/home/gisuser/autocls2/i_20171001'
    ],
    '/home/gisuser/autocls2/fmask2018/fmask_winter_source.tif' : [
        '/home/gisuser/autocls2/i_20180104',
        '/home/gisuser/autocls2/i_20180119'
    ],
    '/home/gisuser/autocls2/fmask2018/fmask_spring_source.tif' : [
        '/home/gisuser/autocls2/i_20180315',
        '/home/gisuser/autocls2/i_20180320'
    ],
    '/home/gisuser/autocls2/fmask2018/fmask_summer_source.tif' : [
        '/home/gisuser/autocls2/i_20180812',
        '/home/gisuser/autocls2/i_20180822'
    ]
}

outfolder = '/home/gisuser/autocls2/resimg'

In [ ]:
import os
from osgeo import gdal, gdal_array
import numpy as np

from glass.prop.img import rst_epsg
from glass.pys.oss import lst_ff, mkdir, fprop
from glass.cons.sat import get_lwibands
from glass.wt.rst import obj_to_rst

In [ ]:
rbands = get_lwibands()

In [ ]:
_data = {}

for k in data:
    _data[k] = []

    for i in data[k]:
        bands = lst_ff(i, file_format='.tif')

        _bands = {}

        for b in rbands:
            for _b in bands:
                if b in _b:
                    _bands[b] = _b
                    break

        _data[k].append(_bands)

In [ ]:
# Produce results
for mask in _data:
    # Make dir to store results
    iout_fld = mkdir(os.path.join(outfolder, fprop(mask, 'fn')), overwrite=True)
    
    # Open Mask
    mask_src = gdal.Open(mask, gdal.GA_ReadOnly)

    mask_nd  = mask_src.GetRasterBand(1).GetNoDataValue()
    mask_num = mask_src.GetRasterBand(1).ReadAsArray()
    _mask_val = list(np.unique(mask_num))

    mask_val = [v for v in _mask_val if v != mask_nd]

    # Create new bands:
    for b in _data[mask][0]:
        # Open Bands
        band_src = [gdal.Open(i[b], gdal.GA_ReadOnly) for i in _data[mask]]

        # Get NoDataValues
        ndvals = [s.GetRasterBand(1).GetNoDataValue() for s in band_src]

        # Get Arrays
        arrays = [s.GetRasterBand(1).ReadAsArray() for s in band_src]

        new_band = np.zeros(
            mask_num.shape,
            gdal_array.GDALTypeCodeToNumericTypeCode(
                band_src[0].GetRasterBand(1).DataType
            )
        )

        # Place values
        for v in mask_val:
            if v == 0:
                np.place(new_band, mask_num == 0, ndvals[0])
                continue
            
            np.copyto(new_band, arrays[v-1], 'no', mask_num == v)

        np.place(new_band, mask_num == mask_nd, ndvals[0])

        # Write file
        new_file = obj_to_rst(
            new_band, os.path.join(iout_fld, f'{b}.tif'),
            band_src[0].GetGeoTransform(),
            rst_epsg(band_src[0]), noData=ndvals[0]
        )

In [ ]:
_data